#### CSC 180 Intelligenct Systems (Spring 2021)

#### Dr. Haiquan Chen, Dept of Computer Scicence

#### California State University, Sacramento

## Lab 6: Evaluating Neural Networks 


# Helpful Functions for Tensorflow (little gems)

The following functions will be used with TensorFlow to help preprocess the data.  They allow you to build the feature vector for a neural network. 

* Predictors/Inputs 
    * Fill any missing inputs with the median for that column.  Use **missing_median**.
    * Encode textual/categorical values with **encode_text_dummy**.
    * Encode numeric values with **encode_numeric_zscore**.
* Output
    * Discard rows with missing outputs.
    * Encode textual/categorical values with **encode_text_index**.
    * Do not encode output numeric values.
* Produce final feature vectors (x) and expected output (y) with **to_xy**.

In [2]:
from collections.abc import Sequence
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import os


# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column. 
    target_type = df[target].dtypes
    target_type = target_type[0] if isinstance(target_type, Sequence) else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    else:
        # Regression
        return df[result].values.astype(np.float32), df[target].values.astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
               * (normalized_high - normalized_low) + normalized_low


# Training with a Test Set with Early Stopping

**Overfitting** occurs when a neural network is trained to the point that it begins to memorize rather than generalize.  

![Training vs Validation Error for Overfitting](https://raw.githubusercontent.com/jeffheaton/t81_558_deep_learning/master/images/class_3_training_val.png "Training vs Validation Error for Overfitting")


### Split data into training and test using train_test_split

In [3]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split

from sklearn import metrics

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping

path = "./data/"
    
filename = os.path.join(path,"iris.csv")    
df = pd.read_csv(filename,na_values=['NA','?'])

species = encode_text_index(df,"species")

x,y = to_xy(df,"species")

# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

model = Sequential()

model.add(Dense(10, input_dim=x.shape[1], activation='relu'))
model.add(Dense(5,activation='relu'))
model.add(Dense(y.shape[1],activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=2, verbose=2, mode='auto')  

# patience: number of epochs with no improvement after which training will be stopped

# The test set is checked during training to monitor progress for early stopping but is never used for gradient descent (model training)

model.fit(x_train, y_train, validation_data=(x_test,y_test), callbacks=[monitor], verbose=2, epochs=1000)  


Epoch 1/1000
4/4 - 0s - loss: 1.2122 - val_loss: 1.2528
Epoch 2/1000
4/4 - 0s - loss: 1.1351 - val_loss: 1.1674
Epoch 3/1000
4/4 - 0s - loss: 1.0653 - val_loss: 1.0895
Epoch 4/1000
4/4 - 0s - loss: 1.0050 - val_loss: 1.0178
Epoch 5/1000
4/4 - 0s - loss: 0.9545 - val_loss: 0.9544
Epoch 6/1000
4/4 - 0s - loss: 0.9055 - val_loss: 0.9027
Epoch 7/1000
4/4 - 0s - loss: 0.8677 - val_loss: 0.8570
Epoch 8/1000
4/4 - 0s - loss: 0.8323 - val_loss: 0.8170
Epoch 9/1000
4/4 - 0s - loss: 0.8021 - val_loss: 0.7818
Epoch 10/1000
4/4 - 0s - loss: 0.7744 - val_loss: 0.7500
Epoch 11/1000
4/4 - 0s - loss: 0.7462 - val_loss: 0.7221
Epoch 12/1000
4/4 - 0s - loss: 0.7230 - val_loss: 0.6991
Epoch 13/1000
4/4 - 0s - loss: 0.7036 - val_loss: 0.6792
Epoch 14/1000
4/4 - 0s - loss: 0.6896 - val_loss: 0.6633
Epoch 15/1000
4/4 - 0s - loss: 0.6777 - val_loss: 0.6499
Epoch 16/1000
4/4 - 0s - loss: 0.6670 - val_loss: 0.6378
Epoch 17/1000
4/4 - 0s - loss: 0.6570 - val_loss: 0.6261
Epoch 18/1000
4/4 - 0s - loss: 0.6484 - 

Now that the neural network is trained, we can make predictions about the test set.  The following code predicts the type of iris for test set and displays the first five irises. 

In [4]:
pred = model.predict(x_test)
print(pred[0:5]) # print first five predictions

[[2.0340462e-03 6.7610937e-01 3.2185659e-01]
 [9.9326444e-01 6.7348885e-03 7.4866131e-07]
 [5.7004744e-08 2.3361864e-02 9.7663808e-01]
 [2.3030471e-03 7.0561779e-01 2.9207918e-01]
 [1.5282637e-03 7.1740502e-01 2.8106681e-01]]


Each line provides the probability that the iris is one of the 3 types of iris in the data set. 

### Saving Best Weights

It would be good idea to keep track of the most optimal weights during the entire training operation.  


An additional monitor, ModelCheckpoint,  is used and saves a copy of the neural network to **best_weights.hdf5** each time the validation score of the neural network improves.  

Once training is done, we just reload this file and we have the optimal training weights that were found.

In [5]:
import pandas as pd
import io
import requests
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

path = "./data/"
    
filename = os.path.join(path,"iris.csv")    
df = pd.read_csv(filename,na_values=['NA','?'])

species = encode_text_index(df,"species")
x,y = to_xy(df,"species")

# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

model = Sequential()
model.add(Dense(10, input_dim=x.shape[1], activation='relu'))
model.add(Dense(5,activation='relu'))
model.add(Dense(y.shape[1],activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')

checkpointer = ModelCheckpoint(filepath="dnn/best_weights.hdf5", verbose=0, save_best_only=True) # save best model

model.fit(x_train, y_train,validation_data=(x_test,y_test),callbacks=[monitor,checkpointer],verbose=2,epochs=1000)

model.load_weights('dnn/best_weights.hdf5') # load weights from best model

Epoch 1/1000
4/4 - 0s - loss: 1.0717 - val_loss: 1.1170
Epoch 2/1000
4/4 - 0s - loss: 1.0527 - val_loss: 1.0994
Epoch 3/1000
4/4 - 0s - loss: 1.0413 - val_loss: 1.0836
Epoch 4/1000
4/4 - 0s - loss: 1.0261 - val_loss: 1.0716
Epoch 5/1000
4/4 - 0s - loss: 1.0180 - val_loss: 1.0598
Epoch 6/1000
4/4 - 0s - loss: 1.0078 - val_loss: 1.0491
Epoch 7/1000
4/4 - 0s - loss: 0.9998 - val_loss: 1.0394
Epoch 8/1000
4/4 - 0s - loss: 0.9923 - val_loss: 1.0305
Epoch 9/1000
4/4 - 0s - loss: 0.9843 - val_loss: 1.0223
Epoch 10/1000
4/4 - 0s - loss: 0.9765 - val_loss: 1.0148
Epoch 11/1000
4/4 - 0s - loss: 0.9720 - val_loss: 1.0072
Epoch 12/1000
4/4 - 0s - loss: 0.9654 - val_loss: 1.0001
Epoch 13/1000
4/4 - 0s - loss: 0.9595 - val_loss: 0.9930
Epoch 14/1000
4/4 - 0s - loss: 0.9542 - val_loss: 0.9856
Epoch 15/1000
4/4 - 0s - loss: 0.9472 - val_loss: 0.9753
Epoch 16/1000
4/4 - 0s - loss: 0.9363 - val_loss: 0.9653
Epoch 17/1000
4/4 - 0s - loss: 0.9244 - val_loss: 0.9568
Epoch 18/1000
4/4 - 0s - loss: 0.9174 - 

OSError: Unable to open file (unable to open file: name = 'dnn/best_weights.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

### Potential Keras Issue on Small Networks Regarding Saving Optimal Weights

You might occasionally see this error:

```
OSError: Unable to create file (Unable to open file: name = 'dnn/best_weights.hdf5', errno = 22, error message = 'invalid argument', flags = 13, o_flags = 302)
```

Usually you can just run rerun the code and it goes away.  This is an unfortnuate result of saving a file each time the validation score improves (as described in the previous section).  If the errors improve two rapidly, you might try to save the file twice and get an error from these two saves overlapping.  For larger neural networks this will not be a problem because each training step will take longer, allowing for plenty of time for the previous save to complete.   

## Evaluating Classification Models

### (1) Calculate Classification Accuracy/Precision/Recall/F1-Score

By default, Keras will return the predicted probability for each class. We can change these prediction probabilities into the actual iris predicted with **argmax**.

In [5]:
pred = model.predict(x_test)
pred

array([[4.71494487e-03, 8.45346868e-01, 1.49938241e-01],
       [9.91973162e-01, 8.02563876e-03, 1.13456713e-06],
       [3.45179842e-06, 1.14592100e-02, 9.88537431e-01],
       [4.64522187e-03, 7.54409015e-01, 2.40945816e-01],
       [3.53244087e-03, 9.27941024e-01, 6.85265660e-02],
       [9.86725211e-01, 1.32721653e-02, 2.55039140e-06],
       [1.58099979e-02, 9.57360506e-01, 2.68294606e-02],
       [6.04713976e-04, 2.81531900e-01, 7.17863441e-01],
       [3.46631696e-03, 7.80912101e-01, 2.15621606e-01],
       [1.09425150e-02, 9.57177758e-01, 3.18797417e-02],
       [8.09577235e-04, 2.81197578e-01, 7.17992783e-01],
       [9.80344415e-01, 1.96491349e-02, 6.47375873e-06],
       [9.89732563e-01, 1.02663469e-02, 1.04882076e-06],
       [9.82253373e-01, 1.77410170e-02, 5.62334071e-06],
       [9.92402792e-01, 7.59436050e-03, 2.86538329e-06],
       [4.05827072e-03, 7.64293194e-01, 2.31648564e-01],
       [2.61457735e-05, 1.92335658e-02, 9.80740309e-01],
       [1.11245075e-02, 9.46039

In [6]:
pred = np.argmax(pred,axis=1) # raw probabilities to choose class (highest probability)
print(pred)

[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0 0 0 1 0 0 2 1
 0]


Now that we have the actual iris flower predicted, we can calculate the percent accuracy (how many were correctly classified).

In [7]:
y_true= np.argmax(y_test,axis=1) 

score = metrics.accuracy_score(y_true, pred)

print("Accuracy score: {}".format(score))

Accuracy score: 1.0


In [8]:
score = metrics.precision_score(y_true, pred, average= "weighted")
print("Precision score: {}".format(score))

Precision score: 1.0


In [9]:
score = metrics.recall_score(y_true, pred, average= "weighted")
print("Recall score: {}".format(score))

Recall score: 1.0


In [10]:
score = metrics.f1_score(y_true, pred, average= "weighted")
print("F1 score: {}".format(score))

F1 score: 1.0


In [11]:
print(metrics.classification_report(y_true, pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        15
           1       1.00      1.00      1.00        11
           2       1.00      1.00      1.00        12

    accuracy                           1.00        38
   macro avg       1.00      1.00      1.00        38
weighted avg       1.00      1.00      1.00        38



### (2) Calculate Classification Cross-Entropy Loss (Log Loss)  

Log loss is an error metric that is often used in place of accuracy for classification.  

Log loss allows for "partial credit". For example, a model might be used to classify A, B and C.  The correct answer might be A, however if the classification network chose B as having the highest probability, then accuracy gives the neural network no credit for this classification.  

However, with log loss, the probability of the correct answer is added to the score.  For example, the correct answer might be A, but if the neural network only predicted .4 probability of A being correct, then the value -log(.4) is added.

$$ logloss = -\frac{1}{N}\sum^N_{i=1}\sum^M_{j=1}y_{ij} \log(\hat{y}_{ij}) $$

The following code shows the logloss scores that correspond to the average probablity for the correct item. The **pred** column specifies the average robability for the correct class.  The **logloss** column specifies the log loss for that probability.


Calculating log loss

In [12]:
# Generate predictions
pred = model.predict(x_test)

print("Numpy array of predictions")
print(pred[0:5])
print()
print("y_test:")
print(y_test[0:5])

score = metrics.log_loss(y_test, pred)
print("Log loss score: {}".format(score))

Numpy array of predictions
[[4.7149449e-03 8.4534687e-01 1.4993824e-01]
 [9.9197316e-01 8.0256388e-03 1.1345671e-06]
 [3.4517984e-06 1.1459210e-02 9.8853743e-01]
 [4.6452219e-03 7.5440902e-01 2.4094582e-01]
 [3.5324409e-03 9.2794102e-01 6.8526566e-02]]

y_test:
[[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]]
Log loss score: 0.100190236594675


## Evaluating Regression Models

Regression results are evaluated differently than classification.  Consider the following code. 

In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn import metrics

path = "./data/"

filename_read = os.path.join(path,"auto-mpg.csv")
df = pd.read_csv(filename_read,na_values=['NA','?'])

cars = df['name']
df.drop('name',1,inplace=True)
missing_median(df, 'horsepower')

encode_text_dummy(df, 'origin')

x,y = to_xy(df,"mpg")

# Split into train/test
x_train, x_test, y_train, y_test = train_test_split (x, y, test_size=0.25, random_state=45)

model = Sequential()
model.add(Dense(10, input_dim=x.shape[1], activation='relu'))
model.add(Dense(10))
model.add(Dense(10))
model.add(Dense(10))

model.add(Dense(1))  # 1 output neuron 


model.compile(loss='mean_squared_error', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')

model.fit(x_train,y_train, validation_data=(x_test,y_test),callbacks=[monitor],verbose=2,epochs=1000)


NameError: name 'missing_median' is not defined

### Mean Square Error

The mean square error is the sum of the squared differences between the prediction ($\hat{y}$) and the expected ($y$).  MSE values are not of a particular unit.  If an MSE value has decreased for a model, that is good. Low MSE values are desired.

$ \text{MSE} = \frac{1}{n} \sum_{i=1}^n \left(\hat{y}_i - y_i\right)^2 $


In [14]:
# Predict
pred = model.predict(x_test)

# Measure MSE error.  
score = metrics.mean_squared_error(pred,y_test)
print("Final score (MSE): {}".format(score))

Final score (MSE): 152.66622924804688


### Root Mean Square Error

The root mean square (RMSE) is essentially the square root of the MSE.  Because of this, the RMSE error is in the same units as the training data outcome. Low RMSE values are desired.

$ \text{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^n \left(\hat{y}_i - y_i\right)^2} $

In [15]:
# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final score (RMSE): {}".format(score))

Final score (RMSE): 12.355817794799805


# Performance Improvement by Normalizing Features and Tuning Hyperparameters

There are many different settings that you can use for a neural network.  These can affect performance.  The following code changes some of these, beyond their default values:

* **activation:** relu, sigmoid, tanh
* **Layers and Neuron Counts**
* **optimizer:** adam, sgd, rmsprop, and [others](https://keras.io/optimizers/)

In [12]:
import pandas as pd
import io
import requests
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

path = "./data/"
preprocess = True

filename_read = os.path.join(path,"auto-mpg.csv")
df = pd.read_csv(filename_read,na_values=['NA','?'])

# create feature vector
missing_median(df, 'horsepower')
encode_text_dummy(df, 'origin')
df.drop('name',1,inplace=True)

if preprocess:
    encode_numeric_zscore(df, 'horsepower')
    encode_numeric_zscore(df, 'weight')
    encode_numeric_zscore(df, 'cylinders')
    encode_numeric_zscore(df, 'displacement')
    encode_numeric_zscore(df, 'acceleration')
    encode_numeric_zscore(df, 'year')

# Encode to a 2D matrix for training
x,y = to_xy(df,'mpg')

# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

model = Sequential()
model.add(Dense(100, input_dim=x.shape[1], activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=2, mode='auto')

model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor],verbose=2,epochs=1000)


Epoch 1/1000
10/10 - 0s - loss: 596.3494 - val_loss: 543.4562
Epoch 2/1000
10/10 - 0s - loss: 547.6803 - val_loss: 500.6264
Epoch 3/1000
10/10 - 0s - loss: 499.6248 - val_loss: 453.0867
Epoch 4/1000
10/10 - 0s - loss: 447.6172 - val_loss: 396.4079
Epoch 5/1000
10/10 - 0s - loss: 397.4583 - val_loss: 352.8488
Epoch 6/1000
10/10 - 0s - loss: 361.7157 - val_loss: 324.5508
Epoch 7/1000
10/10 - 0s - loss: 338.1561 - val_loss: 304.7567
Epoch 8/1000
10/10 - 0s - loss: 320.8242 - val_loss: 290.0575
Epoch 9/1000
10/10 - 0s - loss: 307.4329 - val_loss: 278.0957
Epoch 10/1000
10/10 - 0s - loss: 296.1365 - val_loss: 267.6558
Epoch 11/1000
10/10 - 0s - loss: 285.7850 - val_loss: 258.3828
Epoch 12/1000
10/10 - 0s - loss: 276.6533 - val_loss: 249.6531
Epoch 13/1000
10/10 - 0s - loss: 267.8512 - val_loss: 241.5288
Epoch 14/1000
10/10 - 0s - loss: 259.7338 - val_loss: 233.7492
Epoch 15/1000
10/10 - 0s - loss: 251.9246 - val_loss: 226.3604
Epoch 16/1000
10/10 - 0s - loss: 244.4237 - val_loss: 219.3418
E

In [14]:
# Predict and measure RMSE
pred = model.predict(x_test)
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Score (RMSE): {}".format(score))

Score (RMSE): 2.512824058532715


In [15]:
# print out prediction
df_y = pd.DataFrame(y_test, columns=['ground_truth'])
df_pred = pd.DataFrame(pred, columns=['predicted'])
result = pd.concat([df_y, df_pred],axis=1)
result

,ground_truth,predicted
0,33.0,33.359409
1,28.0,30.762089
2,19.0,21.347250
3,13.0,15.328108
4,14.0,13.172492
...,...,...
75,19.9,19.097708
76,17.5,19.440075
77,28.0,29.027054
78,29.0,28.909090
